<!-- # Processing the final dataset
 -->


# Export data as CSV

In [1]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder
last_year = config.last_year


## Main Model Dataset

### Age, Biomass

In [2]:
# Fire and Land Use
age = ee.Image(f"{data_folder}/mapbiomas_{last_year}")

ESA_CCI = ee.Image(f"projects/sat-io/open-datasets/ESA/ESA_CCI_AGB/CCI_BIOMASS_100m_AGB_{last_year}_v51").select("AGB").rename("biomass")

fire = (ee.Image("projects/mapbiomas-public/assets/brazil/fire/collection3/mapbiomas_fire_collection3_annual_burned_coverage_v1")
    .select([f"burned_coverage_{year}" for year in config.range_1985_2020])
    .byte()
    .rename([str(year) for year in config.range_1985_2020])
    .gt(0)
    .reduce('sum').rename("num_fires")).unmask(0)

floodable_forests = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
        .select(f"classification_{last_year}").eq(6)).rename("floodable_forests")

secondary_area = ee.Image(f"{data_folder}/secondary_area")
pasture_area = ee.Image(f"{data_folder}/pasture_area")


### Surrounding Landscape

In [3]:
quarters_ecoreg_biomass = ee.Image("projects/amazon-forest-regrowth/assets/quarters_ecoreg_biomass")
ecoreg = ee.Image("projects/amazon-forest-regrowth/assets/ecoreg")
distance_deep_forest = ee.Image(f"{data_folder}/distance_deep_forest").rename("dist")
sur_cover = ee.Image(f"{data_folder}/sur_cover")

distance_gt_1000 = distance_deep_forest.gt(1000).selfMask()
mature_biomass = ee.Image(f"{data_folder}/mature_biomass")
mature_biomass_10k = ee.Image(f"{data_folder}/mature_biomass_10k")
nearest_mature = ee.Image(f"{data_folder}/nearest_mature")

### Environmental

In [4]:
categorical = ee.Image(f"{data_folder}/categorical")

topography = ee.Image("CSP/ERGo/1_0/Global/ALOS_landforms").rename("topography") # 90m resolution

soil = ee.Image(f"{data_folder}/soilgrids")

terraclim = ee.Image(f"{data_folder}/terraclim_1958_2019")

In [5]:
# Load each scenario
historical = ee.Image(f"{data_folder}/CMIP6_historical")
ssp126 = ee.Image(f"{data_folder}/CMIP6_ssp126")
ssp245 = ee.Image(f"{data_folder}/CMIP6_ssp245")
ssp585 = ee.Image(f"{data_folder}/CMIP6_ssp585")

CMIP6_means = ee.Image(f"{data_folder}/CMIP6_means")

### Export CSV


In [6]:
# Define reprojection function
def reproject_mean(img):
    return img.reduceResolution(ee.Reducer.mean(), maxPixels=65536).reproject(age.projection())

def reproject_first(img):
    return img.reduceResolution(ee.Reducer.first(), maxPixels=65536).reproject(age.projection())

# Reproject continuous variables using mean
continuous_vars = [
    ESA_CCI.rename("biomass"),
    nearest_mature.rename("nearest_mature"),
    soil,
    terraclim.select(["mean_soil", "mean_vpd", "mean_temp", "mean_def", 
                      "mean_srad", "mean_pr", "mean_aet", "mean_pdsi"])
]

# Reproject categorical variables using first
categorical_vars = [
    fire,
    ecoreg,
    categorical,
    topography
]

# Create unified dataset by directly combining bands
unified_data = ee.Image.cat([
    age,
    floodable_forests,
    distance_deep_forest,
    sur_cover,
    *[reproject_mean(img) for img in continuous_vars],
    *[reproject_first(img) for img in categorical_vars],
    ee.Image.pixelLonLat().rename(['lon', 'lat'])
])

unified_data_pasture = ee.Image.cat([
    unified_data,
    pasture_area.rename('pasture_area')
])

unified_data_secondary = ee.Image.cat([
    unified_data,
    secondary_area,
    reproject_mean(quarters_ecoreg_biomass)
])

Since processing gets too heavy if we try to extract each point at a time for the entire Amazon at 1km resolution, we do it in chunks:

In [7]:
def export_csv(name, image, grid_size, n_chunks = 1, lu_name = None):

    properties_to_export = image.bandNames().getInfo()
    
    # Get FeatureCollection size and calculate chunk size
    grid = ee.FeatureCollection(f"{data_folder}/grid_{grid_size}k_{name}")
    total_features = grid.size().getInfo()
    chunk_size = int(total_features * 1/n_chunks)

    if lu_name:
        name = f"{name}_{lu_name}"

    def process_chunk(chunk_index):
        start = chunk_index * chunk_size
        chunk = grid.toList(chunk_size, start)
        selected_pixels = ee.FeatureCollection(chunk)

        unified_fc = image.reduceRegions(selected_pixels, ee.Reducer.first(), 30)

        task = ee.batch.Export.table.toDrive(
            collection = unified_fc,
            description = f"grid_{grid_size}k_{name}_{chunk_index}",
            fileFormat = "CSV",
            selectors = [p for p in properties_to_export if p not in ['system:index', '.geo']]
        )
        task.start()

    for i in range(n_chunks):
        if i*chunk_size < total_features:
            process_chunk(i)

export_csv("pastureland", unified_data_pasture, 10, n_chunks = 60)
# export_csv("secondary", unified_data_secondary, 1, n_chunks = 30)
# export_csv("secondary", unified_data_secondary, 10, n_chunks = 30)


EEException: Collection.loadTable: Collection asset 'projects/amazon-forest-regrowth/assets/grid_10k_pastureland' not found.

In [ ]:
suffixes = [
    "aggregated_all",
    "non_aggregated_all",
    "non_aggregated_10yr",
    "non_aggregated_5yr"
]

for suffix in suffixes:
    combined_image = ee.Image.cat([
        unified_data, 
        ee.Image(f"{data_folder}/land_use_{suffix}")
    ])
    export_csv("secondary", combined_image, 10, n_chunks=30, lu_name=suffix)


In [ ]:
def export_csv(name, image, grid_size, n_chunks = 1):

    properties_to_export = image.bandNames().getInfo()

    # Get FeatureCollection size and calculate chunk size
    grid = ee.FeatureCollection(f"{data_folder}/grid_{grid_size}k_{name}")
    total_features = grid.size().getInfo()
    chunk_size = int(total_features * 1/n_chunks)

    def process_chunk(chunk_index):
        start = chunk_index * chunk_size
        chunk = grid.toList(chunk_size, start)
        selected_pixels = ee.FeatureCollection(chunk)

        unified_fc = image.reduceRegions(selected_pixels, ee.Reducer.first(), 30)

        task = ee.batch.Export.table.toAsset(
            collection = unified_fc,
            description = f"{grid_size}k_amazon_{name}",
            assetId = f"projects/amazon-forest-regrowth/assets/grids/{name}_{grid_size}k"
        )
        task.start()


    i = 0
    if i*chunk_size < total_features:
        process_chunk(i)

# export_csv("pastureland", unified_data_pasture, 1, n_chunks = 60)
# export_csv("secondary", unified_data_secondary, 1, n_chunks = 30)
export_csv("secondary", unified_data_secondary, 10, n_chunks = 30)


In [31]:
amazon_secondary_10k = ee.FeatureCollection("projects/amazon-forest-regrowth/assets/grids/amazon_secondary_10k")


map = geemap.Map(center=[-3.5, -60.24], zoom=6)
map.addLayer(ESA_CCI, {'min':50, 'max':200, 'palette':['blue', 'red']}, "ESA_CCI", opacity=0.4)
map.addLayer(unified_data, {}, "unified_data", opacity=0.1)
map.addLayer(amazon_secondary_10k, {}, "10k_amazon_secondary", opacity=0.4)
# map.addLayer(amazon_secondary_1k, {}, "1k_amazon_secondary", opacity=0.4)
map

Map(center=[-3.5, -60.24], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

## Different land use aggregations


In [26]:
tst = ee.Image(f"{data_folder}/land_use_non_aggregated_5yr")
selected_pixels = ee.FeatureCollection(f"{data_folder}/grid_10k_secondary")

unified_fc = tst.reduceRegions(selected_pixels, ee.Reducer.first(), 30)

map = geemap.Map()
map.addLayer(tst, {}, "tst", opacity=0.1)
map.addLayer(unified_fc, {}, "unified_fc")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Field Data

In [ ]:
field_data = ee.FeatureCollection(f"{data_folder}/field_biomass")

unified_fc = unified_data.reduceRegions(field_data, ee.Reducer.first(), 30)

# Export task to Google Drive
task = ee.batch.Export.table.toDrive(
    collection = unified_fc,
    description = 'unified_field',
    fileFormat = "CSV"
)
# task.start()